<a href="https://colab.research.google.com/github/Bbeelina/Trash/blob/main/%D0%BB%D0%B0%D0%B12%D0%91%D0%B5%D0%BB%D0%B0%D0%B9%D0%95%D1%80%D0%BA%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import time
from math import floor
import itertools
''' класс, который описывает каждый предмет, который мы можем положить в рюкзак '''
class Item:
    def __init__(self, id, weight, value):
        self.id = id                                                 # уникальный айди предмета
        self.weight = weight                                         # вес предмета
        self.value = value                                           # ценность предмета
        self.ratio = value / weight if weight != 0 else float('inf') # мера качества (используем в 2 апрокс и в методе ветвей и границ)

    def __lt__(self, other):                                         # сравнение
        return self.ratio < other.ratio

''' функция, которая читает инфу из файликов '''
def read_knapsack_problem(file_prefix):
    with open(f"{file_prefix}_c.txt") as f:
        capacity = int(f.readline().strip())      # вместимость рюкзака

    with open(f"{file_prefix}_w.txt") as f:
        weights = list(map(int, f.readlines()))   # вес объектов

    with open(f"{file_prefix}_p.txt") as f:
        profits = list(map(int, f.readlines()))   # прибыль от каждого объекта

    items = [Item(i, weights[i], profits[i]) for i in range(len(weights))] # создаем список объектов
    return items, capacity

''' 2-approx алгоритм '''
def two_approx(items, capacity):
    sorted_items = sorted(items, key=lambda x: x.ratio, reverse=True) # сортируем по наивысшему качеству
    total_weight = 0
    total_value = 0
    solution = [0] * len(items)

    for item in sorted_items:
        if total_weight + item.weight <= capacity:
            solution[item.id] = 1
            total_weight += item.weight
            total_value += item.value

    # проверка самого ценного предмета
    max_single = 0
    max_single_id = -1
    for item in items:
        if item.weight <= capacity and item.value > max_single:
            max_single = item.value
            max_single_id = item.id

    if max_single > total_value:
        solution = [0] * len(items)
        if max_single_id != -1:
            solution[max_single_id] = 1
        return solution, max_single, max_single

    return solution, total_value, total_weight


''' метод динамического программирования на весах '''
def dp_knapsack(items, capacity):
    n = len(items)
    dp = [0] * (capacity + 1) # максимальная ценность, которую можно получить при каком-то весе
    solution = [[] for _ in range(capacity + 1)]

    for i in range(n): #заполняем таблицу дп
        weight = items[i].weight
        value = items[i].value
        for w in range(capacity, weight - 1, -1):
            if dp[w - weight] + value > dp[w]:
                dp[w] = dp[w - weight] + value
                solution[w] = solution[w - weight] + [items[i].id]

    max_value = dp[capacity]
    selected_items = solution[capacity]
    total_weight = sum(items[i].weight for i in selected_items)

    # преобразуем в норм вид
    full_solution = [0] * n
    for item_id in selected_items:
        full_solution[item_id] = 1

    return full_solution, max_value, total_weight

''' метод ветвей и границ :('''
class Node: # вспомогательный класс с инфо о узлах
    def __init__(self, level, profit, weight, bound, taken):
        self.level = level        # уровень в дереве
        self.profit = profit      # суммарная ценность по уровню
        self.weight = weight      # суммарный весна уровне
        self.bound = bound        # граница для узла
        self.taken = taken.copy() # список выбранных предметов
#  счатиает верхнюю границу
def bound(node, items, capacity):
    if node.weight >= capacity:
        return 0

    profit_bound = node.profit
    total_weight = node.weight
    j = node.level + 1
    n = len(items)

    while (j < n) and (total_weight + items[j].weight <= capacity):
        total_weight += items[j].weight
        profit_bound += items[j].value
        j += 1

    if j < n:
        profit_bound += (capacity - total_weight) * items[j].ratio

    return profit_bound

def branch_and_bound(items, capacity):
    items_sorted = sorted(items, key=lambda x: x.ratio, reverse=True)
    n = len(items_sorted)

    queue = []
    root = Node(-1, 0, 0, 0, [])
    queue.append(root)

    max_profit = 0
    best_solution = []

    while queue:
        current = queue.pop(0)

        # пропускаем листья дерева
        if current.level == n - 1:
            continue

        next_level = current.level + 1
        next_item = items_sorted[next_level]

        # включаем предмет
        include_weight = current.weight + next_item.weight
        include_profit = current.profit + next_item.value
        include_taken = current.taken.copy()
        include_taken.append(next_item.id)

        # обновляем лучшее решение
        if include_weight <= capacity and include_profit > max_profit:
            max_profit = include_profit
            best_solution = include_taken
        # вычисляем границу
        include_bound = bound(Node(next_level, include_profit, include_weight, 0, include_taken),
                            items_sorted, capacity)
        # если видим, что граница больше, чем ценность, то ставим узел в очередь
        if include_bound > max_profit:
            queue.append(Node(next_level, include_profit, include_weight, include_bound, include_taken))

        # не включаем объект
        exclude_bound = bound(Node(next_level, current.profit, current.weight, 0, current.taken),
                            items_sorted, capacity)

        if exclude_bound > max_profit:
            queue.append(Node(next_level, current.profit, current.weight, exclude_bound, current.taken))

    # приводим в нужный нам вид
    full_solution = [0] * n
    for item_id in best_solution:
        full_solution[item_id] = 1
    total_weight = sum(items[i].weight for i in range(n) if full_solution[i] == 1)

    return full_solution, max_profit, total_weight

''' PTAS '''
def ptas_knapsack(items, capacity, epsilon=0.5):
    n = len(items)
    k = max(1, int(1 / epsilon))  # Размер подмножества для перебора
    max_value = 0
    best_solution = [0] * n

    sorted_items = sorted(items, key=lambda x: x.ratio, reverse=True)  # cортируем предметы по убыванию

    # перебираем все подмножества из k предметов
    for subset_indices in itertools.combinations(range(n), min(k, n)):
        subset_weight = sum(items[i].weight for i in subset_indices)
        subset_value = sum(items[i].value for i in subset_indices)

        if subset_weight > capacity:
            continue

        current_weight = subset_weight
        current_value = subset_value
        solution = [0] * n
        for i in subset_indices:
            solution[i] = 1

        for item in sorted_items:
            if item.id not in subset_indices and current_weight + item.weight <= capacity:
                solution[item.id] = 1
                current_weight += item.weight
                current_value += item.value

        if current_value > max_value: #обновляем лучшее решение
            max_value = current_value
            best_solution = solution.copy()

    total_weight = sum(items[i].weight for i in range(n) if best_solution[i] == 1)
    return best_solution, max_value, total_weight

def evaluate_algorithm(algorithm, items, capacity, name):
    start_time = time.time()
    solution, value, weight = algorithm(items, capacity)
    end_time = time.time()

    print(f"{name}:")
    print(f"Solution: {solution}")
    print(f"Total value: {value}")
    print(f"Total weight: {weight}")
    print(f"Time: {end_time - start_time:.6f} seconds")
    print()

    return end_time - start_time

def run_problem(problem_prefix):
    print(f"\n*** Problem {problem_prefix} ***")
    items, capacity = read_knapsack_problem(problem_prefix)

    times = []

    time_2approx = evaluate_algorithm(two_approx, items, capacity, "2-Approx")
    times.append(time_2approx)

    time_dp = evaluate_algorithm(dp_knapsack, items, capacity, "Dynamic Programming")
    times.append(time_dp)

    time_bb = evaluate_algorithm(branch_and_bound, items, capacity, "Branch and Bound")
    times.append(time_bb)

    time_ptas = evaluate_algorithm(ptas_knapsack, items, capacity, "PTAS")
    times.append(time_ptas)

    return times

# бенчмарки
problems = ["p01", "p02", "p03", "p04", "p05", "p06", "p07"]
all_times = []

for problem in problems:
    all_times.append(run_problem(problem))

# итоговая табличка
print("\nExecution times:")
print("Problem\t2-Approx\tDP\tBranch&Bound\tPTAS")
for i in range(len(problems)):
    print(f"{problems[i]}\t{all_times[i][0]:.6f}\t{all_times[i][1]:.6f}\t{all_times[i][2]:.6f}\t{all_times[i][3]:.6f}")


*** Problem p01 ***
2-Approx:
Solution: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Total value: 309
Total weight: 165
Time: 0.000016 seconds

Dynamic Programming:
Solution: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Total value: 309
Total weight: 165
Time: 0.000498 seconds

Branch and Bound:
Solution: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Total value: 309
Total weight: 165
Time: 0.000254 seconds

PTAS:
Solution: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Total value: 309
Total weight: 165
Time: 0.000150 seconds


*** Problem p02 ***
2-Approx:
Solution: [1, 0, 1, 0, 0]
Total value: 47
Total weight: 23
Time: 0.000009 seconds

Dynamic Programming:
Solution: [0, 1, 1, 1, 0]
Total value: 51
Total weight: 26
Time: 0.000035 seconds

Branch and Bound:
Solution: [0, 1, 1, 1, 0]
Total value: 51
Total weight: 26
Time: 0.000081 seconds

PTAS:
Solution: [0, 1, 1, 1, 0]
Total value: 51
Total weight: 26
Time: 0.000040 seconds


*** Problem p03 ***
2-Approx:
Solution: [1, 1, 0, 1, 0, 0]
Total value: 146
Total weight: 179
Time: 0.000008 second